In [1]:
from collections import *
from heapq import *
from functools import *
from math import *
import numpy as np
import copy
import time

data = None
wires = []
gates = []

allwires = set()


with open('input.txt', 'r') as file:
    data = file.readlines()
    data = [d.strip().split(' ') for d in data]
    for d in data:
        if len(d) == 5:
            a,gate,b,_,c = d
            gates.append((a,b,gate,c))
            allwires.add(a)
            allwires.add(b)
            allwires.add(c)
        elif len(d)==2:
            a,b = d
            a = a[:-1]
            b = bool(int(b))
            wires.append((a,b))
            allwires.add(a)


#print(wires,gates)
#print(data)

In [2]:
def hierarchy_pos(G, root=None, width=1., vert_gap = 0.2, vert_loc = 0, xcenter = 0.5):

    '''
    From Joel's answer at https://stackoverflow.com/a/29597209/2966723
    Licensed under Creative Commons Attribution-Share Alike 
    
    If the graph is a tree this will return the positions to plot this in a 
    hierarchical layout.
    
    G: the graph (must be a tree)
    
    root: the root node of current branch 
    - if the tree is directed and this is not given, 
      the root will be found and used
    - if the tree is directed and this is given, then 
      the positions will be just for the descendants of this node.
    - if the tree is undirected and not given, 
      then a random choice will be used.
    
    width: horizontal space allocated for this branch - avoids overlap with other branches
    
    vert_gap: gap between levels of hierarchy
    
    vert_loc: vertical location of root
    
    xcenter: horizontal location of root
    '''
    #if not nx.is_tree(G):
      #  raise TypeError('cannot use hierarchy_pos on a graph that is not a tree')

    if root is None:
        if isinstance(G, nx.DiGraph):
            root = next(iter(nx.topological_sort(G)))  #allows back compatibility with nx version 1.11
        else:
            root = random.choice(list(G.nodes))

    def _hierarchy_pos(G, root, width=1., vert_gap = 0.2, vert_loc = 0, xcenter = 0.5, pos = None, parent = None):
        '''
        see hierarchy_pos docstring for most arguments

        pos: a dict saying where all nodes go if they have been assigned
        parent: parent of this branch. - only affects it if non-directed

        '''
    
        if pos is None:
            pos = {root:(xcenter,vert_loc)}
        else:
            pos[root] = (xcenter, vert_loc)
        children = list(G.neighbors(root))
        if not isinstance(G, nx.DiGraph) and parent is not None:
            children.remove(parent)  
        if len(children)!=0:
            dx = width/len(children) 
            nextx = xcenter - width/2 - dx/2
            for child in children:
                nextx += dx
                pos = _hierarchy_pos(G,child, width = dx, vert_gap = vert_gap, 
                                    vert_loc = vert_loc-vert_gap, xcenter=nextx,
                                    pos=pos, parent = root)
        return pos

            
    return _hierarchy_pos(G, root, width, vert_gap, vert_loc, xcenter)

In [3]:
known = {wire:val for wire,val in wires}
#print(known)
unknown = set([wire for wire in allwires if wire not in known])
#print(unknown)
#print(gates)
currgates = copy.deepcopy(gates)
while len(unknown)>0:
   # print(unknown)
    newgates = []
    for gate in currgates:
        a,b,typ,c = gate
        #print(a,b,typ,c)
        if c in unknown and a in known and b in known:
           # print('here')
            val = -1
            a = known[a]
            b = known[b]
            if typ == 'AND':
                val = a and b
            elif typ == "OR":
                val = a or b
            elif typ == "XOR":
                val = a ^ b
            assert val != -1
            unknown.remove(c)
            known[c] = val
        else:
            newgates.append(gate)
    currgates = newgates
zknown = [(wire,val) for wire,val in known.items() if wire.startswith('z')]
zknown.sort()
zstring = ''.join([str(int(b)) for a,b in zknown])
zstring = int(zstring[::-1],2)
print(zstring)


36035961805936


In [4]:
import matplotlib.pyplot as plt
import networkx as nx
from networkx.drawing.nx_agraph import graphviz_layout
import graphviz
G=nx.DiGraph()

name_to_int = {}
int_to_name = {}
wireset = set([w for a,b,g,c in gates for w in [a,b,c]])
for i,name in enumerate(wireset):
    name_to_int[name] = i
    int_to_name[i] = name
zwires = []
for wire in wireset:
    if wire.startswith('z'):
        zwires.append((-1,name_to_int[wire]))
networkxedges = [(name_to_int[c],name_to_int[w]) for a,b,g,c in gates for w in [a,b]]
networkxedges.extend(zwires)
print(networkxedges)
G.add_edges_from(networkxedges)
pos = graphviz_layout(G, prog='dot')

# Draw the graph with matplotlib
plt.figure(figsize=(8, 6))
nx.draw(tree, pos, with_labels=True, arrows=True, node_size=3000, node_color='lightblue', font_size=10, font_color='black')
plt.title("Directed Tree Graph")
plt.show()

[(77, 152), (77, 67), (2, 29), (2, 273), (52, 308), (52, 297), (233, 128), (233, 295), (90, 120), (90, 278), (38, 23), (38, 138), (57, 188), (57, 222), (46, 132), (46, 99), (279, 8), (279, 35), (154, 43), (154, 240), (143, 28), (143, 115), (300, 95), (300, 279), (187, 283), (187, 76), (135, 93), (135, 183), (254, 82), (254, 11), (185, 159), (185, 217), (102, 309), (102, 59), (183, 166), (183, 270), (4, 169), (4, 176), (158, 134), (158, 117), (112, 146), (112, 214), (257, 91), (257, 48), (199, 211), (199, 50), (68, 38), (68, 193), (238, 234), (238, 124), (189, 151), (189, 242), (258, 249), (258, 241), (56, 29), (56, 273), (218, 89), (218, 42), (9, 83), (9, 218), (142, 176), (142, 169), (285, 20), (285, 198), (171, 248), (171, 4), (43, 296), (43, 26), (260, 258), (260, 25), (139, 84), (139, 251), (16, 22), (16, 116), (256, 51), (256, 272), (208, 231), (208, 126), (178, 215), (178, 206), (149, 203), (149, 207), (145, 123), (145, 221), (266, 43), (266, 240), (280, 211), (280, 50), (180, 20

ImportError: requires pygraphviz http://pygraphviz.github.io/

In [ ]:
children = {c:(a,b) for a,b,g,c in gates}
gateType = {c:(g) for a,b,g,c in gates}
def getNodes(node,gatesOnly=True):
    if node not in children:
        if gatesOnly:
            return set()
        return set([node])
    totalnodes = set([node])
    for child in children[node]:
        totalnodes |= getNodes(child,gatesOnly)
    return totalnodes

def getGates(node):
    return getNodes(node,True)
def getWires(node):
    return getNodes(node,False)
def getInputs(node):
    s = getChildren(node)
    return set([n for n in s if n.startswith('x') or n.startswith('y')])

inputSize = 45
outputSize = 46

childDescendants = [(i,getInputs('z'+"%02d"%(i,))) for i in range(outputSize)]
childDescendants.sort()
print(childDescendants)

outputGates = [getGates('z'+"%02d"%(i,)) for i in range(outputSize)]
outputWires = [getWires('z'+"%02d"%(i,)) for i in range(outputSize)]

In [ ]:
def solve(node,knownwires,knowngates,knowninputs):
    nodewires = getWires(node)
    nodegates = getGates(node)
    nodeinputs = getInputs(node)

    unknowngates = nodegates not in knowngates
    unknownwires = nodewires not in knownwires
    unknowninputs = nodewires not in knowninputs
    
    
    
    known = {wire:val for wire,val in wires}
    #print(known)
    unknown = set([wire for wire in allwires if wire not in known])
    #print(unknown)
    #print(gates)
    currgates = copy.deepcopy(gates)
    while len(unknown)>0:
       # print(unknown)
        newgates = []
        for gate in currgates:
            a,b,typ,c = gate
            #print(a,b,typ,c)
            if c in unknown and a in known and b in known:
               # print('here')
                val = -1
                a = known[a]
                b = known[b]
                if typ == 'AND':
                    val = a and b
                elif typ == "OR":
                    val = a or b
                elif typ == "XOR":
                    val = a ^ b
                assert val != -1
                unknown.remove(c)
                known[c] = val
            else:
                newgates.append(gate)
        currgates = newgates
    zknown = [(wire,val) for wire,val in known.items() if wire.startswith('z')]
    zknown.sort()
    zstring = ''.join([str(int(b)) for a,b in zknown])
    zstring = int(zstring[::-1],2)
    print(zstring)
    
    
    